##part b) Neural network with pytorch with linear modules-high level primitives

---



---






In [1]:
import numpy as np
import random
from numpy import array
from numpy.random import uniform
from numpy import hstack
import torch
import torch.nn as nn

In [2]:

n, d = 400, 3
num_hidden1=5 # number of neurons for first hidden layer
num_hidden2=4 # numbe rof neurons for second hidden layer
x = torch.FloatTensor(n, d).uniform_(-1, 1)
weights_true = torch.tensor([[5,1,1],[4,1,1]]).float()
weights_true = torch.transpose(weights_true,0,1)
bias_true = torch.tensor([1])
y_true = torch.mm(x**2,weights_true) + torch.mm(x,weights_true) + bias_true
print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')


x: torch.Size([400, 3]), weights: torch.Size([3, 2]), bias: torch.Size([1]), y: torch.Size([400, 2])


"\n# Generating data of 3 dim input and 2 dim output\nn, d = 500, 3\nx = torch.FloatTensor(n, d).uniform_(-1, 1)\nweights_true = torch.tensor([[5,1,5],[1,2,1]]).float()\nweights_true = torch.transpose(weights_true,0,1)\nbias_true = torch.tensor([1,2])\ny_true = torch.mm(x**2,weights_true) + torch.mm(x,weights_true) + bias_true\nprint(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')\n"

In [3]:
# Custom Model using nn.Linear modules
class TorchModel(nn.Module):
  def __init__(self, input_dim, num_hidden):
    super().__init__()
    self.linear1 = nn.Linear(input_dim, num_hidden)
    self.relu1 = nn.ReLU()
    self.linear2 = nn.Linear(num_hidden, 12)
    self.relu2 = nn.ReLU()
    self.linear3 = nn.Linear(12, 2)
  
  def forward(self, x):
    l1 = self.linear1(x)
    r1 = self.relu1(l1)
    l2 = self.linear2(r1)
    r2 = self.relu2(l2)
    l3 = self.linear3(r2)
    return l3

In [4]:
# Now we run the training loop
from typing import Callable

def torch_fit(x, y, model: Callable, loss: Callable, lr: float, num_epochs: int):
  optimizer = torch.optim.SGD(model.parameters(), lr=lr)
  for epoch in range(num_epochs):
    optimizer.zero_grad() # Initialize gradient as zero
    y_pred = model(x) # Forward Pass
    loss_value = loss(y_pred, y) # Compute loss with MSE
    if epoch % 50 == 0:
      print(f'Epoch {epoch}, loss {loss_value}')
    loss_value.backward() # Use autogradient to compute backward pass
    optimizer.step() # Update weights 

loss = nn.MSELoss()
model = TorchModel(d, 22)
torch_fit(x, y_true, model=model, loss=loss, lr=0.035, num_epochs=1000)

Epoch 0, loss 19.68140411376953
Epoch 50, loss 0.5106377601623535
Epoch 100, loss 0.220831960439682
Epoch 150, loss 0.1393679678440094
Epoch 200, loss 0.10817503184080124
Epoch 250, loss 0.09145838767290115
Epoch 300, loss 0.08012136816978455
Epoch 350, loss 0.07137203961610794
Epoch 400, loss 0.06445906311273575
Epoch 450, loss 0.05824574455618858
Epoch 500, loss 0.052025437355041504
Epoch 550, loss 0.04700518772006035
Epoch 600, loss 0.04234974831342697
Epoch 650, loss 0.038505446165800095
Epoch 700, loss 0.03523585945367813
Epoch 750, loss 0.03246453404426575
Epoch 800, loss 0.030269937589764595
Epoch 850, loss 0.028306473046541214
Epoch 900, loss 0.02674555592238903
Epoch 950, loss 0.025534339249134064


In [5]:
def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()


In [6]:
from sklearn.manifold import TSNE
X_reduced = TSNE(n_components=2).fit_transform(x)
y_true_reduced = TSNE(n_components=1).fit_transform(y_true)
y_pred_reduced = TSNE(n_components=1).fit_transform(model(x).detach())
print(f'X_reduced: {X_reduced.shape}, y_true_reduced: {y_true_reduced.shape}, y_pred_reduced: {y_pred_reduced.shape}')

plot_intereactive_3d(X_reduced,y_true_reduced,y_pred_reduced)

X_reduced: (400, 2), y_true_reduced: (400, 1), y_pred_reduced: (400, 1)
